In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
from process import * 
from tqdm.notebook import tqdm
import seaborn as sns

# データ保存先
data_folda = '../data/'

* masterからデータを追加

In [2]:

### master data から加えるデータを定義
add_list=['name',"prefecture_name",'head_branch','opened_on', 'created_at','genre_first_name', 
          'genre_second_name','genre_third_name','northlatitude', 'eastlongitude',
          'net_reservation_flg','price_range_lunch_owner','price_range_dinner_owner', 
          'price_range_lunch_user','price_range_dinner_user',
          # 20240312追加
          'all_photo_count','photo_food_count','photo_drink_count',
          'photo_interior_count','photo_exterior_count','official_info_flg','max_visit_month']

# make_timeseries_data.ipynb で作成したデータを読み込む
pseudo_time_series=pd.read_pickle(data_folda+'skeleton_train.pkl')

# 直近三カ月の閉店率を計算して保存
# pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-08-31']
# close_ratios=calculate_close_ratio(pseudo_time_series, data_folda)
# close_ratios=pd.read_pickle(data_folda+'close_ratios.pkl')

# 三カ月の閉店率を読み込み
close_ratios=pd.read_pickle(data_folda+'close_ratios.pkl')

# 2023-01-01からの閉店を予測する。base_date が 2023-01-01の一日前以後が対象
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
# base_date が 2023年1月1日以降のデータを抽出 (closed_date がそれ以降の閉店店舗しかクロールしていないため)
pseudo_time_series = add_master_data(pseudo_time_series, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series=pd.merge(pseudo_time_series,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series.to_pickle(data_folda+'train_with_master.pkl')

pseudo_time_series_test3=pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test3 = add_master_data(pseudo_time_series_test3, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test3=pd.merge(pseudo_time_series_test3,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master.pkl')

pseudo_time_series_test6=pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test6 = add_master_data(pseudo_time_series_test6, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test3=pd.merge(pseudo_time_series_test6,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_master.pkl')

* V_GOOGLE からデータを追加

In [3]:
pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series = add_v_google(pseudo_time_series, data_folda, add_list)
pseudo_time_series.to_pickle(data_folda+'skeleton_train_with_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_google.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_RETTY からデータを追加

In [7]:
#追加する変数をリストで定義
add_list=["restaurant_id",'infos.isOfficial', 
       'infos.familiar.users.count', 'infos.familiar.stars',
       'infos.onlineReservation_RETTY', 'infos.access.transferTime1','infos.access.transferTime2','infos.access.transferTime3',
       'infos.numberOfSeats.value', 'infos.counterSeats.value',
       'infos.privateRoom.value', 'infos.updateInfo.firstReviewDate',
       'infos.wantToGo', 'infos.updateInfo.lastReviewDate',
       'infos.went', 'infos.rateByWent', 'infos.excellent', 'infos.coupon',
       'infos.photoCount.Photograph', 'infos.photoCount.Cooking',
       'infos.photoCount.Interior', 'infos.photoCount.Exterior',
       'infos.photoCount.Menu', 'infos.reviewCount.Review',
       'infos.reviewCount.Lunch', 'infos.reviewCount.Dinner',
       'menus.course_list_count',
       'menus.dishes_list_count', 'menus.lunch_list_count',
       'menus.drink_list_count', 'menus.takeout_list_count',
       'infos.familiar.users.genge','infos.reserveEntireStore',"general_rank_RETTY"]
 
pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')         
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series = add_v_retty(pseudo_time_series, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_retty.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* HOTPEPPER からデータを追加

In [8]:
add_list=['restaurant_id','infos.aggregateRating','infos.ratingReview',
          'infos.satisfaction.percentage1','infos.satisfaction.percentage2','infos.satisfaction.percentage3',
          'infos.couponCount','infos.onlineReservation_HOTPEPPER','infos.ownerRegistration','infos.ticket',
          'photos.allCount','photos.appearanceCount','photos.drinkCount','photos.foodCount','photos.otherCount',
          'photos.postCount','menus.courseListCount']


pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series= reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_hotpepper(pseudo_time_series, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_hotpepper.pkl')

pseudo_time_series_test3 = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test3= reduce_mem_usage(pseudo_time_series_test3)
pseudo_time_series_test3 = add_v_hotpepper(pseudo_time_series_test3, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_hotpepper.pkl')

pseudo_time_series_test6 = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test6= reduce_mem_usage(pseudo_time_series_test6)
pseudo_time_series_test6 = add_v_hotpepper(pseudo_time_series_test6, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_hotpepper.pkl')

del pseudo_time_series, pseudo_time_series_test3, pseudo_time_series_test6

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_STRUCTURED からデータを取得

In [4]:
add_list = '"restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount", \
    "aggregateRating.ratingValue", "aggregateRating.reviewCount", \
    "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went", \
    "interactionCount.wanttogo","paymentAccepted"'


pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series = add_v_structured(pseudo_time_series, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True)
pseudo_time_series.to_pickle(data_folda+'train_with_structured.pkl')

del pseudo_time_series

pseudo_time_series_test3 = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test3 = add_v_structured(pseudo_time_series_test3, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True) 
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_structured.pkl')

pseudo_time_series_test6 = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test6 = add_v_structured(pseudo_time_series_test6, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True) 
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_structured.pkl')

del pseudo_time_series_test3, pseudo_time_series_test6

select "restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount",     "aggregateRating.ratingValue", "aggregateRating.reviewCount",     "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went",     "interactionCount.wanttogo","paymentAccepted" from KRAKEN_DG.KRAKEN_USER.V_STRUCTURED


/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

select "name","hostname" from KRAKEN_DG.KRAKEN_USER.V_MEDIA_SITE_LIST
www.instagram.com
sarah30.com
demae-can.com
www.ekiten.jp
www.facebook.com
www.tripadvisor.jp
www.google.co.jp
tokyo-calendar.jp
autoreserve.com
rtrp.jp
www.navitime.co.jp
www.jalan.net
ramendb.supleks.jp
retty.me
www.hotpepper.jp
gourmet.aumo.jp
www.ubereats.com
twitter.com
loco.yahoo.co.jp
paypaygourmet.yahoo.co.jp
hitosara.com
4travel.jp
place.line.me
restaurant.ikyu.com
tabelog.com
www.ozmall.co.jp
r.gnavi.co.jp
select "restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount",     "aggregateRating.ratingValue", "aggregateRating.reviewCount",     "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went",     "interactionCount.wanttogo","paymentAccepted" from KRAKEN_DG.KRAKEN_USER.V_STRUCTURED


KeyboardInterrupt: 

* V_TRIPADVISOR から特徴量を追加

In [12]:
add_list=["restaurant_id","general_rank_TRIPADVISOR",'infos.aggregateRating_TRIPADVISOR',"infos.reviewCount","infos.qa_count"]

pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series = reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_tripadvisor(pseudo_time_series, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_tripadvisor.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_tripadvisor.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_tripadvisor.pkl')

del pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_HITOSARA から特徴量を追加

In [2]:
# hitosara premium を追加
add_list=["restaurant_id","infos.premium","general_rank_HITOSARA","infos.onlineReservation_HITOSARA"]

pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series= reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_hitosara(pseudo_time_series, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_hitosara.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_hitosara.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_hitosara.pkl')

del pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

In [4]:
#train_with_hotpepper.pkl      
#train_with_google.pkl  
#train_with_master.pkl         
#train_with_retty.pkl          
#train_with_tripadvisor.pkl
# 上記五つを pd.concatで結合。 train_with_master 以外は、base_date と restaurant_id が重複するため、あらかじめ削除しておく
df_list=[]
concat_list=["train_with_master.pkl","train_with_hotpepper.pkl","train_with_google.pkl","train_with_retty.pkl","train_with_tripadvisor.pkl"
             ,"train_with_hitosara.pkl"]
#pseudo_time_series=pseudo_time_series.drop(["base_date","restaurant_id"],axis=1)
for i, fn in enumerate(concat_list):
    print(fn)
        
    df=pd.read_pickle(data_folda+fn)
    if(i==0):
        continue
    else:
        df=df.drop(["base_date","restaurant_id"],axis=1)
    print(df.info())
    df_list.append(df)

pseudo_time_series=pd.concat(df_list,axis=1)
pseudo_time_series.to_pickle(data_folda+'train_with_all.pkl')

train_with_master.pkl
train_with_hotpepper.pkl
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10503768 entries, 0 to 10503767
Data columns (total 27 columns):
 #   Column                             Dtype   
---  ------                             -----   
 0   general_rank_HOTPEPPER             object  
 1   infos.aggregateRating              float64 
 2   infos.ratingReview                 float64 
 3   infos.satisfaction.percentage1     float64 
 4   infos.satisfaction.percentage2     float64 
 5   infos.satisfaction.percentage3     float64 
 6   infos.couponCount                  object  
 7   infos.onlineReservation_HOTPEPPER  category
 8   infos.ownerRegistration            category
 9   infos.ticket                       category
 10  photos.allCount                    object  
 11  photos.appearanceCount             object  
 12  photos.drinkCount                  object  
 13  photos.foodCount                   object  
 14  photos.otherCount                  object  
 15  

In [10]:
df_list=[]
concat_list=["skeleton_test_3month_master.pkl","skeleton_test_3month_hotpepper.pkl","skeleton_test_3month_google.pkl","skeleton_test_3month_retty.pkl","skeleton_test_3month_tripadvisor.pkl"
             ,"skeleton_test_3month_hitosara.pkl"]
#pseudo_time_series=pseudo_time_series.drop(["base_date","restaurant_id"],axis=1)
for i, fn in enumerate(concat_list):
    print(fn)
    
    df=pd.read_pickle(data_folda+fn)
    if(i==0):
        continue
    else:
        df=df.drop(["base_date","restaurant_id"],axis=1)
    print(df.info())
    df_list.append(df)

pseudo_time_series=pd.concat(df_list,axis=1)
pseudo_time_series.to_pickle(data_folda+'test3_with_all.pkl')

skeleton_test_3month_master.pkl


skeleton_test_3month_hotpepper.pkl
<class 'pandas.core.frame.DataFrame'>
Int64Index: 844322 entries, 0 to 844321
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype   
---  ------                             --------------   -----   
 0   general_rank_HOTPEPPER             501132 non-null  object  
 1   infos.aggregateRating              39579 non-null   float64 
 2   infos.ratingReview                 39579 non-null   float64 
 3   infos.satisfaction.percentage1     49886 non-null   float64 
 4   infos.satisfaction.percentage2     57805 non-null   float64 
 5   infos.satisfaction.percentage3     43534 non-null   float64 
 6   infos.couponCount                  501132 non-null  object  
 7   infos.onlineReservation_HOTPEPPER  501132 non-null  category
 8   infos.ownerRegistration            501132 non-null  category
 9   infos.ticket                       501132 non-null  category
 10  photos.allCount                    501132 non-null  objec

In [11]:
df_list=[]
concat_list=["skeleton_test_6month_master.pkl","skeleton_test_6month_hotpepper.pkl","skeleton_test_6month_google.pkl","skeleton_test_6month_retty.pkl","skeleton_test_6month_tripadvisor.pkl"
             ,"skeleton_test_6month_hitosara.pkl"]
#pseudo_time_series=pseudo_time_series.drop(["base_date","restaurant_id"],axis=1)
for i, fn in enumerate(concat_list):
    print(fn)
    
    df=pd.read_pickle(data_folda+fn)
    if(i==0):
        continue
    else:
        df=df.drop(["base_date","restaurant_id"],axis=1)
    print(df.info())
    df_list.append(df)

pseudo_time_series=pd.concat(df_list,axis=1)
pseudo_time_series.to_pickle(data_folda+'test6_with_all.pkl')

skeleton_test_6month_master.pkl


skeleton_test_6month_hotpepper.pkl
<class 'pandas.core.frame.DataFrame'>
Int64Index: 835153 entries, 0 to 835152
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype   
---  ------                             --------------   -----   
 0   general_rank_HOTPEPPER             497782 non-null  object  
 1   infos.aggregateRating              39275 non-null   float64 
 2   infos.ratingReview                 39275 non-null   float64 
 3   infos.satisfaction.percentage1     49341 non-null   float64 
 4   infos.satisfaction.percentage2     57183 non-null   float64 
 5   infos.satisfaction.percentage3     43058 non-null   float64 
 6   infos.couponCount                  497782 non-null  object  
 7   infos.onlineReservation_HOTPEPPER  497782 non-null  category
 8   infos.ownerRegistration            497782 non-null  category
 9   infos.ticket                       497782 non-null  category
 10  photos.allCount                    497782 non-null  objec

In [10]:
pseudo_time_series.columns

Index(['general_rank_HOTPEPPER', 'infos.aggregateRating', 'infos.ratingReview',
       'infos.satisfaction.percentage1', 'infos.satisfaction.percentage2',
       'infos.satisfaction.percentage3', 'infos.couponCount',
       'infos.onlineReservation_HOTPEPPER', 'infos.ownerRegistration',
       'infos.ticket', 'photos.allCount', 'photos.appearanceCount',
       'photos.drinkCount', 'photos.foodCount', 'photos.otherCount',
       'photos.postCount', 'menus.courseListCount', 'infos.fanCount',
       'infos.reviewTags.count1', 'infos.reviewTags.count2',
       'infos.reviewTags.count3', 'infos.reviewTags.count4',
       'infos.reviewTags.count5', 'infos.reviewTags.count6',
       'infos.reviewTags.count7', 'infos.reviewTags.count8',
       'infos.reviewTags.count9', 'overview.aggregateRating',
       'overview.reviewCount', 'general_rank_GOOGLE', 'infos.isOfficial',
       'infos.familiar.users.count', 'infos.familiar.stars',
       'infos.onlineReservation_RETTY', 'infos.access.transferTi

In [7]:
# pseudo_time_series_filter[general_rank_columns]をプロット
# 横軸を S,A,B,L,F,N,NaNの分布
# 一つのキャンバスにプロット
general_rank_columns=["general_rank_HOTPEPPER","general_rank_GOOGLE","general_rank_HITOSARA","general_rank_RETTY","general_rank_TRIPADVISOR"]
pseudo_time_series_filter=pseudo_time_series[general_rank_columns]
fig, ax = plt.subplots()

for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].unique())
    # 前のプロットを上書きする
    ax.hist(pseudo_time_series_filter[column])

plt.show()
"""
for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].unique())
    # 前のプロットを上書きする
    pseudo_time_series_filter[column].value_counts().plot(kind='bar',title=column,ax=plt.gca())
    plt.show()
"""


KeyError: "['general_rank_HITOSARA', 'general_rank_TRIPADVISOR'] not in index"

In [ ]:
skeleton_train["general_rank_HITOSARA"]

0           S
1           S
2           S
3           S
4           S
           ..
52518835    S
52518836    S
52518837    S
52518838    S
52518839    S
Name: general_rank_HITOSARA, Length: 52518840, dtype: object

In [ ]:
skeleton_train=pd.read_pickle(data_folda+'skeleton_train.pkl')
skeleton_train=pd.merge(skeleton_train,pseudo_time_series_filter[["restaurant_id"]+general_rank_columns],on="restaurant_id",how="left")
skeleton_train.to_pickle(data_folda+'skeleton_train_with_general_rank.pkl')

In [ ]:
skeleton_test=pd.read_pickle(data_folda+'skeleton_test.pkl')
skeleton_test=pd.merge(skeleton_test,pseudo_time_series_filter[["restaurant_id"]+general_rank_columns],on="restaurant_id",how="left")
skeleton_test.to_pickle(data_folda+'skeleton_test_with_general_rank.pkl')

NameError: name 'pseudo_time_series_filter' is not defined

In [ ]:
pseudo_time_series_filter[pseudo_time_series_filter["general_rank_HITOSARA"].isin(["A","B","S","L","N","F"])]["general_rank_HITOSARA"].value_counts()

S    546677
A     50424
L     34853
B      1130
F        25
N         1
Name: general_rank_HITOSARA, dtype: int64

In [ ]:
g_filter=pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter[g_filter["general_rank_RETTY"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_HOTPEPPER"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_HITOSARA"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_TRIPADVISOR"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_STRUCTURED"].isin(["A","B","S","L","N","F"])]

print(g_filter.describe())


        original_id        target3        target6            id  \
count  8.694380e+05  869438.000000  869438.000000  8.694380e+05   
mean   2.083029e+07       0.007363       0.013388  2.083029e+07   
std    1.199797e+07       0.085494       0.114929  1.199797e+07   
min    1.000001e+06       0.000000       0.000000  1.000001e+06   
25%    1.303462e+07       0.000000       0.000000  1.303462e+07   
50%    2.000145e+07       0.000000       0.000000  2.000145e+07   
75%    2.801496e+07       0.000000       0.000000  2.801496e+07   
max    4.703118e+07       1.000000       1.000000  4.703118e+07   

       northlatitude  eastlongitude  net_reservation_flg  \
count  868633.000000  868633.000000        869438.000000   
mean       35.507799     137.054747             0.082691   
std         2.559043       3.995540             0.275415   
min         0.000000       0.000000             0.000000   
25%        34.673365     135.416475             0.000000   
50%        35.437477     138.207717 

In [ ]:
pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]["general_rank_HITOSARA"].value_counts()

S    544765
A     50302
L     34550
B      1130
F        25
N         1
Name: general_rank_HITOSARA, dtype: int64

In [ ]:
ggoogle=pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]["general_rank_GOOGLE"].value_counts()

In [ ]:
rank_num=pd.DataFrame()
for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].dropna().shape[0])
    #上記を転置してrank_numに追加
    rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])

general_rank_GOOGLE 869438
general_rank_RETTY 637495


/tmp/ipykernel_7303/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])


general_rank_HOTPEPPER 517620
general_rank_STRUCTURED 294864


/tmp/ipykernel_7303/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])
/tmp/ipykernel_7303/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])


general_rank_TRIPADVISOR 413841
general_rank_HITOSARA 633110


/tmp/ipykernel_7303/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])


In [ ]:
rank_num.sort_values(by=1,ascending=False)

,0,1
0,general_rank_GOOGLE,869438
0,general_rank_RETTY,637495
0,general_rank_HITOSARA,633110
0,general_rank_HOTPEPPER,517620
0,general_rank_TRIPADVISOR,413841
0,general_rank_STRUCTURED,294864


In [ ]:
rank_num=pd.DataFrame()
for column in general_rank_columns:
    num=pseudo_time_series_filter[pseudo_time_series_filter["closed_date"].isna()==False][column].dropna().shape[0]
    ratio=num/len(pseudo_time_series_filter[pseudo_time_series_filter["closed_date"].isna()==False])
    print(column,num,ratio)
    #上記を転置してrank_numに追加
    rank_num=rank_num.append([[column,num,ratio]])
rank_num

general_rank_GOOGLE 18153 0.9909383699983624


/tmp/ipykernel_7303/1519989912.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


general_rank_RETTY 13491 0.7364484960969485
general_rank_HOTPEPPER 9424 0.5144385610568263
general_rank_STRUCTURED 10434 0.5695725749222119


/tmp/ipykernel_7303/1519989912.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


general_rank_TRIPADVISOR 8679 0.4737704023145368


/tmp/ipykernel_7303/1519989912.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


general_rank_HITOSARA 2532 0.13821715159124406


/tmp/ipykernel_7303/1519989912.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


In [ ]:
rank_num=pd.DataFrame()
for column in general_rank_columns:
    num=pseudo_time_series_filter[column].dropna().shape[0]
    ratio=num/len(pseudo_time_series_filter)
    print(column,num,ratio)
    #上記を転置してrank_numに追加
    rank_num=rank_num.append([[column,num,ratio]])
rank_num

general_rank_GOOGLE 869438 0.9932869804435894
general_rank_RETTY 637495 0.7283043570650075
general_rank_HOTPEPPER 517620 0.5913535028572604
general_rank_STRUCTURED 294864 0.3368665416067834


/tmp/ipykernel_7303/1641020988.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])
/tmp/ipykernel_7303/1641020988.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


general_rank_TRIPADVISOR 413841 0.4727914782580879
general_rank_HITOSARA 633110 0.7232947262353853


/tmp/ipykernel_7303/1641020988.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])
/tmp/ipykernel_7303/1641020988.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


,0,1,2
0,general_rank_GOOGLE,869438,0.993287
0,general_rank_RETTY,637495,0.728304
0,general_rank_HOTPEPPER,517620,0.591354
0,general_rank_STRUCTURED,294864,0.336867
0,general_rank_TRIPADVISOR,413841,0.472791
0,general_rank_HITOSARA,633110,0.723295


In [ ]:
pseudo_time_series_filter["closed_date"].dropna().shape[0]

18319

In [ ]:
rank_num.sort_values(by=1,ascending=False)

,0,1
0,general_rank_GOOGLE,18153
0,general_rank_RETTY,13491
0,general_rank_STRUCTURED,10434
0,general_rank_HOTPEPPER,9424
0,general_rank_TRIPADVISOR,8679
0,general_rank_HITOSARA,2532


In [ ]:
g_filter=pseudo_time_series_filter
rlist=["A","B","S","L","N","F"]
rlist=["A","B","S"]
#num_restaurants=pd.DataFrame()
for column in rank_num.sort_values(by=1,ascending=False)[0].values:
    if(column=="general_rank_STRUCTURED"):
        continue
    g_filter=g_filter[g_filter[column].isin(rlist)]
    num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])

/tmp/ipykernel_7303/543598693.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])
/tmp/ipykernel_7303/543598693.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])
/tmp/ipykernel_7303/543598693.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])
/tmp/ipykernel_7303/543598693.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

In [ ]:
# general_rank_GOOGLE, general_rank_RETTY, general_rank_HOTPEPPER, general_rank_HITOSARA, general_rank_TRIPADVISORそれぞれがある無しで、どの程度のレストランが抽出されるかを確認

# 各メディアの true False を保存する dataframe を作成
num_restaurants=pd.DataFrame()

#rlist=["A","B","S","L","N","F"]
rlist=["A","B","S"]

for is_google in [True,False]:
    for is_retty in [True,False]:
        for is_hotpepper in [True,False]:
            for is_hitosara in [True,False]:
                for is_tripadvisor in [True,False]:
                    g_filter=pseudo_time_series_filter
                    if is_google:
                        g_filter=g_filter[g_filter["general_rank_GOOGLE"].isin(rlist)]
                    if is_retty:
                        g_filter=g_filter[g_filter["general_rank_RETTY"].isin(rlist)]
                    if is_hotpepper:
                        g_filter=g_filter[g_filter["general_rank_HOTPEPPER"].isin(rlist)]
                    if is_hitosara:
                        g_filter=g_filter[g_filter["general_rank_HITOSARA"].isin(rlist)]
                    if is_tripadvisor:
                        g_filter=g_filter[g_filter["general_rank_TRIPADVISOR"].isin(rlist)]
                    print(is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0])
                    
                    num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])

True True True True True 81058


/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])


True True True True False 166415
True True True False True 88447
True True True False False 191736


/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])


True True False True True 118365


...

/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])


False False False True False 598231


/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])
/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])


False False False False True 199718
False False False False False 875314


/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])


In [ ]:
#num_restaurants　の is_google, is_retty, is_hotpepper, is_hitosara, is_tripadvisor を0,1,2,3,4から代入
num_restaurants.columns=["is_google","is_retty","is_hotpepper","is_hitosara","is_tripadvisor","num_restaurants"]

In [ ]:
num_restaurants

,is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,num_restaurants
0,True,True,True,True,True,251776
0,True,True,True,True,False,366712
0,True,True,True,False,True,287423
0,True,True,True,False,False,435798
0,True,True,False,True,True,311464
0,True,True,False,True,False,512371
0,True,True,False,False,True,361836
0,True,True,False,False,False,634156
0,True,False,True,True,True,269153
0,True,False,True,True,False,411548


In [ ]:
#num_restaurants=num_restaurants.drop([0,1,2,3,4,5],axis=1)
num_restaurants["num_restaurants"]=num_restaurants["num_restaurants"].astype(int)
num_restaurants.sort_values(by="num_restaurants",ascending=False,inplace=True)
num_restaurants[["is_google","is_retty","is_hotpepper","is_hitosara","is_tripadvisor","num_restaurants"]]
num_restaurants


,is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,num_restaurants
0,False,False,False,False,False,875314
0,False,False,False,True,False,598231
0,True,False,False,False,False,573939
0,False,True,False,False,False,451540
0,True,False,False,True,False,418996
0,False,True,False,True,False,359697
0,True,True,False,False,False,354547
0,False,False,True,False,False,302134
0,True,True,False,True,False,291669
0,True,False,True,False,False,242427
